<a href="https://colab.research.google.com/github/EmmanuelADAM/IntelligenceArtificiellePython/blob/master/TPMLGymQLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning
## Appliqué à [Gym.OpenAI](https://gym.openai.com)

---
## Présentation de Gym

Voir la page d'introduction à [Gym](https://gym.openai.com).

### Installation de gym 

#### Outil AUTRE QUE COLAB (pyzo, jupyter lab, .....)

In [ ]:
!pip install gymnasium

In [2]:
import gymnasium as gym

---
## Test de ML par Q-Learning pour atteindre l'objectif
- Utiliser l'environnement `FrozenLake8x8-v1` (un labyrinthe en mode texte)
- 4 actions sont possibles (Left(0), Down(1), Right(2), Up(3))
  - l'adjectif "Frozen" signifie qu'une *action n'est pas déterministe !*
    - à partir d'une case "gelée", aller à droite peut .. mener à droite, ou pas
    - => intérêt du Q-Learning adapté à ce type d'environnement probabiliste
- Le labyrinthe est ainsi composé de zones glacées, de puits, et d'un objectif

En vous basant sur les codes présent dans l'article [Q-Learning en Java](http://emmanuel.adam.free.fr/site/spip.php?article134), programmer un algo de Q-Learning pour apprendre à atteindre l'objectif 

**N.B.** 
  - *Cet environnement fonctionne bien sous colab, jupyterlab.. quelques soucis de l'affichage de l'état courant (carré rouge) sous Pyzo....* 
  - Il est fortement conseillé de débuter avec un environnement déterministe pour évaluer la bonne marche de l'algo de Q-Learning que vous aurez développer..
    - pour cela, lancer l'environnement avec 

---
#### Etude de l'environnement

In [14]:
env = gym.make('FrozenLake-v1', map_name="4x4", is_slippery=True, render_mode='ansi') 
# tester map_name="8x8" pour l'environnement plus large
print(env.spec)
print(env.action_space) #ici 4 actions discrétisée
print(env.observation_space) # ici 4x4 cellules possibles

env.reset()
print(env.render())
print("S = Start, G = Goal, H = Hole, F = Frozen place")

EnvSpec(id='FrozenLake-v1', entry_point='gymnasium.envs.toy_text.frozen_lake:FrozenLakeEnv', reward_threshold=0.7, nondeterministic=False, max_episode_steps=100, order_enforce=True, autoreset=False, disable_env_checker=False, apply_api_compatibility=False, kwargs={'map_name': '4x4', 'is_slippery': True, 'render_mode': 'ansi'}, namespace=None, name='FrozenLake', version=1)
Discrete(4)
Discrete(16)

SFFF
FHFH
FFFH
HFFG

S = Start, G = Goal, H = Hole, F = Frozen place


---
#### Test des actions

En mode non déterministe, chaque action a 1 chance sur 3 de réussir..<br>
*Exemple aller à droite mène 1/3 à droite, 1/3 en haut, 1/3 en bas.*

In [19]:
###### Test des fonctions
env.reset()
action = 0
observation, reward, done, _, info = env.step(action)
print(env.render())
print("pos° actuelle:", observation,",gain:", reward,",fini:", done,",", info)

  (Left)
SFFF
FHFH
FFFH
HFFG

pos° actuelle: 4 ,gain: 0.0 ,fini: False , {'prob': 0.3333333333333333}


In [20]:
env.reset()
action = 1
observation, reward, done, _, info = env.step(action)
print(env.render())
print("pos° actuelle:", observation,",gain:", reward,",fini:", done,",", info)

  (Down)
SFFF
FHFH
FFFH
HFFG

pos° actuelle: 4 ,gain: 0.0 ,fini: False , {'prob': 0.3333333333333333}


In [21]:
env.reset()
action = 2
observation, reward, done, _, info = env.step(action)
print(env.render())
print("pos° actuelle:", observation,",gain:", reward,",fini:", done,",", info)

  (Right)
SFFF
FHFH
FFFH
HFFG

pos° actuelle: 0 ,gain: 0.0 ,fini: False , {'prob': 0.3333333333333333}


In [22]:
env.reset()
action = 3
observation, reward, done, _, info = env.step(action)
print(env.render())
print("pos° actuelle:", observation,",gain:", reward,",fini:", done,",", info)

  (Up)
SFFF
FHFH
FFFH
HFFG

pos° actuelle: 0 ,gain: 0.0 ,fini: False , {'prob': 0.3333333333333333}


In [23]:
#note : 
## observation = position où se trouve l'agent (no case(i,j) = i*largeur+j)
## reward = recompense
## done = but atteint
## truncated = etat feuille (non utilise ici)
## info = proba de reussite de l'action

On est clairement ici dans un environnement non déterministe (une même action à partir d'un même état ne mène pas toujours au même résultat); c'est le contexte de prédilection de l'algo de Q-Learning...

---
### <font color="red">Mise en place de l'environnement en mode déterministe</font>
Important, pour valider l'apprentissage de votre algorithme avant de passer en mode non-déterministe, il vaut mieux le tester sur un environnement où chaque action à 100% de réussite. Ci-dessous un exemple sur le mini labyrinthe.

In [24]:
env = gym.make('FrozenLake-v1', map_name="4x4", is_slippery=False, render_mode='ansi') 


In [26]:
actions = [2,2,1,1,1,2]
env.reset()
for a in actions:
    observation, reward, done,_, info = env.step(a)
    print(env.render())
    print("pos° actuelle:", observation,",gain:", reward,",fini:", done,",", info)


  (Right)
SFFF
FHFH
FFFH
HFFG

pos° actuelle: 1 ,gain: 0.0 ,fini: False , {'prob': 1.0}
  (Right)
SFFF
FHFH
FFFH
HFFG

pos° actuelle: 2 ,gain: 0.0 ,fini: False , {'prob': 1.0}
  (Down)
SFFF
FHFH
FFFH
HFFG

pos° actuelle: 6 ,gain: 0.0 ,fini: False , {'prob': 1.0}
  (Down)
SFFF
FHFH
FFFH
HFFG

pos° actuelle: 10 ,gain: 0.0 ,fini: False , {'prob': 1.0}
  (Down)
SFFF
FHFH
FFFH
HFFG

pos° actuelle: 14 ,gain: 0.0 ,fini: False , {'prob': 1.0}
  (Right)
SFFF
FHFH
FFFH
HFFG

pos° actuelle: 15 ,gain: 1.0 ,fini: True , {'prob': 1.0}


---
***Aide Python : (plusieurs possibilités)***
- Il n'est pas nécessaire de créer des classes Etat, ....
- Il faut pouvoir stocker les récompenses de chaque actions à partir de chaque case
  - a priori, la création d'une matrice 4x4 de 4 valeurs peut être utile.
  Plus simplement, on utilisera un tableau de n cases (pour les n etats) contenant m valeurs d'actions 
  

In [18]:
import numpy as np
# cree un tableau de n cases x m actions
q_actions = np.zeros((env.observation_space.n, env.action_space.n), np.float32)
print(q_actions)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


*Recherche du max et de sa position*

Reprenons la derniere modélisation sous forme de tableau de 4 valeurs.

In [21]:
#Exemple de recherche..
#1. ici pour l'exemple, on place des valeurs aléatoires pour les actions
Q = np.random.random((env.observation_space.n, env.action_space.n))
print(Q)

[[0.01297509 0.90311764 0.36200217 0.53466891]
 [0.21753126 0.1486138  0.69545345 0.39602564]
 [0.18348307 0.49858799 0.84607384 0.66118449]
 [0.4971295  0.49150569 0.56847206 0.12887271]
 [0.52758408 0.53058904 0.94523054 0.47123764]
 [0.78793628 0.58430868 0.49208643 0.92930246]
 [0.69077113 0.66702524 0.22414255 0.23459581]
 [0.35539074 0.54387106 0.77523979 0.40162692]
 [0.05791587 0.02296204 0.68227821 0.54090457]
 [0.00854811 0.26819492 0.96686916 0.60920238]
 [0.86511296 0.34100465 0.19701795 0.90531074]
 [0.48291018 0.06886919 0.28706745 0.58680669]
 [0.32912292 0.95590134 0.19309512 0.93499283]
 [0.44193389 0.95742732 0.24923035 0.52057711]
 [0.00340703 0.69191096 0.76140266 0.98655288]
 [0.42682983 0.90372205 0.03912378 0.61866962]]


In [22]:
#max de la case 2
max_2 = np.max(Q[2, :])
position_max_2 = np.argmax(Q[2,:])
print("case 2, max=",max_2, ", en position ", position_max_2)
print("la meilleure action de l'etat 2 serait donc ", position_max_2)

case 2, max= 0.8460738379472967 , en position  2
la meilleure action de l'etat 2 serait donc  2


  -----
# Développer la solution en Q-Learning

Pour rappel, l'algorithme de Q-Learning est le suivant : 

- Initialiser la table des poids des arcs :
  - $\forall s \forall a Q(s,a) \gets 0$ 
- Pour n boucles d'apprentissage
  - $\lambda \gets 1 ; \epsilon \gets 1 ;$
  - $etat Courant \gets  etat initial$
  - Pour n' itérations ( n' = nombre max supposé d'actions pour atteindre l'objectif)
    - $s \gets  etat Courant$
    - $nb \gets random(0,1)$
    - Si $nb < \epsilon$ Alors
      - Choisir aléatoirement une action a
    - Sinon
      - Choisir action la plus intéressante : $a \gets argMax_{a'}(Q(s,a'))$
    - Fin Si
    - Effectuer action a :
    - $s' \gets  nouvel Etat Courant$
    - calculer $Q(s,a) \gets \lambda \times (r + \gamma \times max_{a'}(Q(s', a')) + (1-\lambda ) \times Q(s,a)$
    - {{decrémenter}} les coefficients d'apprentissage et d'exploration :
      - $\lambda \gets 0.99 \times \lambda$
      - $\epsilon \gets 0.99 \times \epsilon$
    - Si but atteint, casser la boucle n' (uniquement celle là)
  - Fin pour n' itérations
- Fin pour n boucles d'apprentissage  

En reprenant les définitions précédentes, $Q(s,a)$ est équivalent à `Q[s][a]`

## Initialisation des variables

On utilisera initialement les affectations suivantes pour les variables : 
- variables d'exploration, de récompense et d'apprentissage : 
  - $ \lambda \gets 0.8$
  - $ \gamma \gets 0.998$
- nombre de cycles : 
  - $ n \gets 10000$ si grille 4x4, 50000 si grille 8x8
  - $ n' \gets 10$ si grille 4x4, 20 si grille 8x8
    
----    

 #### Q-Learning simple

In [ ]:
##algorithme de Q-Learning simple
def q_learn(nb_actions):
    """
    effectue un cycle d'apprentissage/recherche de solution' via le Q-Learning simple

    Parameters
    ----------
    epoch : no de l'etape

    Returns
    -------
    total_r : recompense totale
    r : recompense du dernier etat rencontre
    states_list : liste des etats traverses
    actions_list : liste des actions effectuees

    """
    #A DEFINIR


In [ ]:
def try_qlearn(nb_episodes = 4000, nb_actions = 64):
    """
    lance nb_episodes fois un cycle de Q-Learning et memorise chaque solution trouvee

    Returns
    -------
    solutions_list : liste des solutions (no, recompense totale, liste des etats, liste des actions)
    """
    """
    lance nb_episodes fois un cycle de Q-Learning et memorise chaque solution trouvee

    Returns
    -------
    solutions_list : liste des solutions (no, recompense totale, liste des etats, liste des actions)
    """
    global epsilon
    states_list = []
    actions_list = []
    solutions_list = []
    epsilon = 1
    for i in range(nb_episodes):
        # Reset environment and get first new observation
        total_r, r, states_list, actions_list = q_learn(nb_actions)
        epsilon = epsilon * 0.999
        # memorize if a solution has been found
        if r == 1: solutions_list.append((i, total_r, states_list, actions_list))
        
    if(len(solutions_list) == 0): print("aucune solution trouvee !!")

    return solutions_list


----
## Affichage de du résultat
Affichons maintenant la liste des actions via l'environnement Gym

In [ ]:
def rendu(solutions_list):
    """ affiche la plus courte sequence d'actions permettant d'atteindre l'objectif q partir des solutions fournies
    Parameters
    ----------
    solutions_list : liste des solutions trouvees

    Returns
    -------
    None.
    """
    mini_sol = solutions_list[0]
    for s in  solutions_list:
        if len(s[2]) < len(mini_sol[2]): mini_sol = s
    print("une solution en ", len(mini_sol[2]), " etapes : ")
    env.reset()
    env.render()
    for i in range(0, len(mini_sol[2])):
        env.env.s = mini_sol[2][i]
        print("action ", actions[mini_sol[3][i]])
        print(env.render())

In [ ]:
##TEST
##ON LANCE LA RESOLUTION : 
solutions = try_qlearn(3000, 50)
if(len(solutions)>0):rendu(solutions)
#relancer le bloc si pas de solution trouvee

----
**Etudions l'historique**

In [ ]:
import matplotlib.pyplot as plt
def plot_frequence_sol(solutions_list):
    """
    dessine la frequence de solution trouvees
    Parameters
    ----------
    solutions : liste des solutions

    Returns
    -------
    None.

    """
    xs = [x[0] for x in solutions_list]
    ys = [y[1] for y in solutions_list]
    plt.figure(figsize=(10, 4))
    plt.plot(xs, ys, '.')
    plt.show()

In [ ]:
plot_frequence_sol(solutions)

---
## L'algorithme de Double Q-Learning

#### Explication du double Q-Learning
L'algo de **double** Q-Learning simple repose sur DEUX matrices nb_etats x nb_actions; la mise à jour des valeurs d'une action dans une grille utilise la valeur de la meilleure action suivante dans l'autre matrice.

On réalise, plus ou moins alternativement :

$a^* \gets  argmax_{a} QA(s', a)$,

$QA(s,a) \gets QA(s,a) + \lambda \times (r + \gamma \times QB(s', a^*))-  QA(s,a))$ 

ou

$b^* \gets  argmax_{a} QB(s', a)$

$QB(s,a) \gets QB(s,a) + \lambda \times (r + \gamma \times QA(s', b^*))-  QB(s,a))$ 

avec
  - $\lambda$ : coef. d'apprentissage
  - $\gamma$ : coef. de réduction 
  - $r$ : récompense
  
Cette équation donne la qualité de l'action *a* à partir de l'état *s*.

Initialement, à chaque état, une action est choisie aléatoirement (car toutes "valent" 0); puis au fil des tests les actions les plus valuées sont choisies. 

*Vous pouvez voir en fin de cette page des aides sur les fonctionnalités Python utiles pour la manipulation de tableaux.*

#### Initialisation

In [ ]:
# initialiser la Q-Table
# autant de cases que l'environnement en possède, 
# contenant autant de valeurs que d'actions possibles
# donc ici une matrice 64 x 4
QA = np.zeros([env.observation_space.n,env.action_space.n])
QB = np.zeros([env.observation_space.n,env.action_space.n])

lambda_learn = .3
gamma = 0.9

#### Travail à faire
- Codez le double q learning dans une fonction proche de celle décrite pour le Q-Learning.
- Reprenez la suite des fonctions développée pour lancer une résolution en mode déterministe et non déterministe. 
- Comparez les performances  (1ere solution trouvée, fréquence des solutions, ...)
- reprenez au choix le simple QLearning, ou le Double Q-Learning et appliquez l'algorithme sur l'environnement CliffWalking-v0 (point de départ en x, arrivée en T, coût de -1 par action sur o, -100 par action sur C).
   
```o  o  o  o  o  o  o  o  o  o  o  o```

```o  o  o  o  o  o  o  o  o  o  o  o```

```o  o  o  o  o  o  o  o  o  o  o  o```

```x  C  C  C  C  C  C  C  C  C  C  T```
